In [ ]:
!wget 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
!wget 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

# Others
import os

In [ ]:
df = pd.read_parquet('./yellow_tripdata_2023-01.parquet')

In [ ]:
df.head()

In [ ]:
df.info()

## Quesion 1

In [ ]:
df.columns.shape[0]

## Quesion 2

In [ ]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime   
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.head()

In [ ]:
df.duration.std()

## Quesion 3

In [ ]:
sns.boxplot(df.duration)

In [ ]:
def remove_nulls(df):
    no_records = df.shape[0]
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df_clean = df.shape[0]
    clean_prec = df_clean * 100/no_records
    return clean_prec

In [ ]:
remove_nulls(df)

## Quesion 4

In [ ]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape[1]

## Quesion 5

In [ ]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

## Quesion 6

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_val = read_dataframe('./yellow_tripdata_2023-02.parquet')
df_train = read_dataframe('./yellow_tripdata_2023-01.parquet')
                          
print(len(df_train), len(df_val))
                          

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
                          
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)


In [ ]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values


In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)